1. Read a pdf
2. One agent  reads the PDF content and skill set
3. Once agent reads the candidate details and match the jobs which suites with the candidate
4. Another agent use tools to generate message ( uses three diferent tools to generate mails) and then pick the best one . then handoffs to another agent
5. Another agent uses tools to write subject and convert it to HTML and sends mail to candidate. Handoff to another agent
6. That agent will wait for candidate's reply and as soon as candidate reply , it will use tools to generate a mail for real Human HR and schedue an interview and also sends to candidate


In [3]:
# import required libraries
from openai import AsyncOpenAI
from pypdf import PdfReader
from dotenv import load_dotenv
import os
from agents import Agent, Runner,trace,OpenAIChatCompletionsModel,function_tool
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail, Email, To, Content


In [4]:
#load from .env file and override if any existing property 
load_dotenv(override=True)

True

In [5]:
# Read from candidate resume
reader = PdfReader("me/linkedin_arpan.pdf")
candidatedata = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        candidatedata += text

In [6]:
candidatedata

'\xa0 \xa0\nContact\narpan.bandyopadhyay.statist\nics@gmail.com\nwww.linkedin.com/in/arpan-\nbandyopadhyay-bb5b1a54\n(LinkedIn)\ndev.to/arpanforgeek (Blog)\nTop Skills\nDeep Learning\nLarge Language Models (LLM)\nOLLAMA\nLanguages\nEnglish (Full Professional)\nHindi (Limited Working)\nbengali (Native or Bilingual)\nCertifications\nOCA(1ZO-808)\nJavascript\n1Z0-815 Oracle Java 11\nProgrammer\nMasterclass on API Development\nusing Node.js \nPost Graduate Program in Artificial\nIntelligence and Machine Learning\nARPAN BANDYOPADHYAY\nPostgraduate in Artificial Intelligence and Machine Learning .\n(Statistician & Data Science (AI/ML) aspirant.)\nBengaluru, Karnataka, India\nSummary\nAt HCLSoftware, my journey as a Senior Software Engineer III is\ncentered around integrating AI/ML capabilities into high-quality\nJEE-based applications. With a deep understanding of machine\nlearning algorithms, data analytics, and statistical modeling, I focus\non developing intelligent solutions that enhance

In [7]:
# Create a resume reader Agent
resume_read_instruction= f"You are a resume reader agent working for Autozenic. you are given the resume content {candidatedata} , \
your's responsibility is to read the whole resume content and understand it."

# Create instance of Agent (Resume Reader)
resume_reader_agent =  Agent (
    name = "Resume Reader",
    instructions = resume_read_instruction,
    model ="gpt-4o-mini"
)

In [8]:
# Function to read resume by executing Resume Reader Agent
async def read_resume():
    input = "Read the full resume and Get the core skill sets of the candidate."
    skillset = await Runner.run(resume_reader_agent,input)
    return skillset.final_output

In [10]:
# List of available jobs in company - Autozenic
job_list = [
    {
        "title": "Machine Learning Engineer",
        "department": "AI/ML",
        "skills_required": ["Python", "TensorFlow", "Scikit-learn", "Data Science", "Model Deployment"],
        "experience_required": "2+ years"
    },
    {
        "title": "Data Analyst",
        "department": "Analytics",
        "skills_required": ["SQL", "Python", "Excel", "Tableau", "Data Visualization"],
        "experience_required": "1+ years"
    },
    {
        "title": "Frontend Developer",
        "department": "Engineering",
        "skills_required": ["JavaScript", "React", "CSS", "HTML", "REST APIs"],
        "experience_required": "2+ years"
    },
    {
        "title": "Backend Developer",
        "department": "Engineering",
        "skills_required": ["Python", "Django", "PostgreSQL", "APIs", "Docker"],
        "experience_required": "3+ years"
    },
    {
        "title": "DevOps Engineer",
        "department": "Infrastructure",
        "skills_required": ["AWS", "Docker", "Kubernetes", "CI/CD", "Terraform"],
        "experience_required": "3+ years"
    },
    {
        "title": "HR Generalist",
        "department": "Human Resources",
        "skills_required": ["Employee Relations", "Recruiting", "Compliance", "Onboarding", "HRIS"],
        "experience_required": "2+ years"
    },
    {
        "title": "Product Manager",
        "department": "Product",
        "skills_required": ["Roadmapping", "Agile", "Stakeholder Management", "Market Research", "Analytics"],
        "experience_required": "4+ years"
    }
]


# Instruction for Job Alignment Agent to align relavent job for the candidate
role_alignment_instruction = f"You are a Job Matcher Agent working for Autozenic. You are provided with a list of available jobs: {job_list} , \
Your task is to carefully analyze the candidate’s skills, experience, and background, and select the most suitable single job from the list , \
Ensure your matches are relevant and well-justified.  "

# Create instance of ob Alignment Agent
job_alignment_agent = Agent(
    name="Job Alignment Agent",
    instructions=role_alignment_instruction,
    model = "gpt-4o-mini"
)

In [11]:
# Function to align job for the candidate by executing job_alignment_agent
async def align_jobs(skills:str):
    input = f" Based on the following skills: {skills}, identify the most suitable job for the candidate from the available list ,\
    Consider skill relevance, industry fit, and overall alignment with job requirements. Only suggest the best-matched role."
    matching_jobs = await Runner.run(job_alignment_agent,input)
    return matching_jobs.final_output

In [12]:
#Instruction to write a professional email for the candidate with the matching job details 
email_writer_instruction = f"""
You are an Email Writing Agent working at Autozenic.

You are provided with the candidate's details: {candidatedata}.

Your task is to draft a professional and engaging email to the candidate. The goal is to express interest in their profile and share a job opportunity that aligns with their background.

The email should:
- Be polite, clear, and professionally worded
- Briefly highlight how the candidate's experience matches the role
- Include the job description or a summary
- Encourage the candidate to respond if interested.
"""


google_api_key=os.getenv('GOOGLE_API_KEY')
gemini_client = AsyncOpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/",api_key=google_api_key)

gemini_model=OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

# Create instance of Email Writer
email_writter_agent = Agent(
    name="Email Writer",
    instructions=email_writer_instruction,
    model=gemini_model
)

# Converting agents to Tool
email_writter_tool = email_writter_agent.as_tool(tool_name="Email_Writer", tool_description="Generate an email")

In [61]:
# instruction for HR  agent to update HR about the selection of the resume of the candidate 
hr_update_instruction = f"""
You are an HR Update Agent working for Autozenic.

You are provided with the candidate's details: {candidatedata}.

Your responsibility is to notify the human HR team that a job opportunity email has been successfully triggered and sent to the candidate.

Your update should:
- Be clear, professional, and concise
- Include the candidate's name and relevant details if available
- Mention the job role that was shared with the candidate
- Confirm that the email has been sent
- Optionally mention that the system will follow up automatically if no response is received
- Be addressed to internal HR (not the candidate)

Keep the tone business-professional and suitable for internal updates.
"""

# Create instance of HR Agent
hr_update_agent = Agent(
    name="HR Agent",
    instructions=hr_update_instruction,
    model="gpt-4o-mini"
)

# Convert HR agent to tool
hr_update_tool = hr_update_agent.as_tool(tool_name="HR_Agent",tool_description="Send mail to HR/Recruiter")

In [51]:
# Create a function to send email to the candidate and using @function_tool  registering it as a tool
@function_tool
def send_email(body: str):
    print("calling send email")
    sg = SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("arpan1601@gmail.com")
    to_email = To("arpantestengineer@gmail.com")
    content = Content("text/html", body)
    mail = Mail(from_email, to_email, "Job-oppurtunity", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}


In [52]:
# Combine all tools together as a list
text_email_tools = [email_writter_tool,hr_update_tool]

In [16]:
text_email_tools

[FunctionTool(name='Email_Writer', description='Generate an email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Email_Writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000258627E6480>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='HR_Agent', description='Send mail to HR/Recruiter', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'HR_Agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000258627E76A0>, strict_json_schema=True, is_enabled=True)]

In [53]:
# instruction to convert text email to HTML
html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

# Create instance of HTML converter Agent
html_converter_agent = Agent(
    name="HTML converter Agent",
    instructions=html_instructions,
    model="gpt-4o-mini"
)

# Convert HTML converter Agent to tool
html_converter_tool = html_converter_agent.as_tool(tool_name="HTML_converter_Agent",tool_description="Convert text mail to HTML")

In [54]:
convert_and_send_email_tools = [html_converter_tool,send_email]

In [55]:
# Instruction on how to use convert text email to html and send to the candidate
email_manager_instruction = "You are an email formatter and sender. You receive the body of an email to be sent. \
You  use the html_converter_tool to convert the body to HTML. \
After that finally, you use the send_email tool to send the email with the subject and HTML body."

# Create instance of Email  Manager agent
email_composition_agent = Agent(
    name="Email Manager",
    instructions=email_manager_instruction,
    model="gpt-4o-mini",
    tools=convert_and_send_email_tools,
    handoff_description="Convert an email to HTML and send it" #handoff_description helps other agents know when to delegate a task to this agent.
)


In [56]:
# instruction for Candidate Outreach Coordinator
candidate_outreach_cooridinator_instruction = "You are a Alice , working as a Candidate Outreach Coordinator for Autozenic. You use the tools given to you to generate  emails. \
You never generate email by yourself; you always use the tools. \
You generate email multiple times if you're not satisfied with the results from the first try .\
Finally , you handoff to the Email Manager agent to convert it to HTML and send it."

# Create instance of Candidate Outreach Coordinator agent
candidate_outreach_cooridinator_agent = Agent(
    name = "Candidate Outreach Coordinator",
    instructions=candidate_outreach_cooridinator_instruction,
    tools=text_email_tools,
    handoffs=[email_composition_agent],
    model="gpt-4o-mini"
)

In [57]:
# Function to execure candidate_outreach_cooridinator_agent
async def candidate_coorination():
    input = "Send out a professional email addressed candidate and also send an update the HR Team regarding this."
    result =await Runner.run(candidate_outreach_cooridinator_agent,input)
    return result  

In [62]:
# This function orchestrates the end-to-end candidate outreach workflow.
# It performs the following steps:
# 1. Reads and analyzes the candidate's resume to extract core skills.
# 2. Matches the candidate to the most suitable job role based on those skills.
# 3. Generates and sends a professional email to both the candidate and the HR team.
# The function returns a confirmation message once all steps are successfully completed.
async def process():
    with trace("Recruiting service"):
        skills = await read_resume()
        await align_jobs(skills)
        await candidate_coorination()

    return "The Candidate has been contacted and the same has been updated to the HR team."


In [63]:
# This asynchronous chat handler interprets user input and triggers the candidate outreach workflow.
# If the message contains both "check the resume" and "connect", it calls the process() function,
# which performs resume analysis, job alignment, and sends the outreach email.
# Otherwise, it returns a predefined response indicating supported commands.
async def chat(message, history):
    user_input = message.strip().lower()
    if "check the resume" in user_input and "connect" in user_input:
        summary = await process()
        return summary
    else:
        return "I can only process commands like: 'check the resume and connect to the candidate'."


In [60]:
import gradio as gr
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


calling send email
calling send email
calling send email
calling send email
